In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Este notebook require Tensorflow 2.X

In [2]:
jokes = ""

joke_url = "https://raw.githubusercontent.com/Daronspence/One-Liners/master/jokes.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()

jokes += joke_text

joke_url = "http://www.textfiles.com/humor/TAGLINES/taglines.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()
    
jokes += joke_text

joke_url = "https://raw.githubusercontent.com/simonaco/25daysofserverless/master/jokes.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()
    
jokes += joke_text

jokes = jokes.replace("\n", " ")
jokes = jokes.replace("$", " ")
jokes = jokes.replace('"', " ")
jokes = jokes.replace('%', " ")
jokes = jokes.replace('&', " ")
jokes = jokes.replace("'", " ")
jokes = jokes.replace("-", " ")
jokes = jokes.replace(";", "")
jokes = jokes.replace(":", "")
jokes = jokes.replace("=", "")
jokes = jokes.replace(",", "")
    
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(jokes)
max_id = len(tokenizer.word_index) 

In [3]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [4]:
#model = keras.models.load_model('shakespeare-stateless.h5')
model = keras.models.load_model('jokes.h5')

In [5]:
X_new = preprocess(["what is"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # primera oracion ultimo caracter.

' '

In [6]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

La temperatura es un hiperparámetro de los LSTM (y las redes neuronales en general) que se utiliza para controlar la aleatoriedad de las predicciones escalando los logits antes de aplicar softmax.

Cuando la temperatura es 1, calculamos el softmax directamente en los logits (la salida sin escala de capas anteriores)

https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally

In [7]:
tf.random.set_seed(42)

next_char("How are yo", temperature=0.6)

'u'

In [8]:
def complete_text(text, n_chars=300, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
print(complete_text("my mom", temperature=0.1))